In [1]:
using ITensors
using Random

# Effective Overlap Trial

We will start with a small-scale test using only 5 sites initially:

In [3]:
s = siteinds("S=1/2", 5); # create five sites

Construct the test MPS and product state:

In [4]:
Random.seed!(42)
psi = randomMPS(s; linkdims=4) # use arbitary bond dimension χ=4

MPS
[1] ((dim=2|id=567|"S=1/2,Site,n=1"), (dim=4|id=168|"Link,l=1"))
[2] ((dim=4|id=168|"Link,l=1"), (dim=2|id=581|"S=1/2,Site,n=2"), (dim=4|id=12|"Link,l=2"))
[3] ((dim=4|id=12|"Link,l=2"), (dim=2|id=944|"S=1/2,Site,n=3"), (dim=4|id=235|"Link,l=3"))
[4] ((dim=4|id=235|"Link,l=3"), (dim=2|id=489|"S=1/2,Site,n=4"), (dim=2|id=56|"Link,l=4"))
[5] ((dim=2|id=56|"Link,l=4"), (dim=2|id=65|"S=1/2,Site,n=5"))


To make things simpler, we will construct a product state manually because ITensor represents product states as an MPS with bond dimension 1. These additional 'bonds' can make things confusing. By manually constructing a product state as a vector of ITensors (each corresponding only to the physical site) we can get around these annoying additional bonds indices:

In [5]:
phi = Vector{ITensor}(undef, length(s)); # container for physical sites
for i=1:length(phi)
    phi[i] = randomITensor(s[i])
end
# convert to a "MPS" data type so we can use ITensor operations on the product state
phi = MPS(phi)
normalize!(phi)

MPS
[1] ((dim=2|id=567|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=581|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=944|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=489|"S=1/2,Site,n=4"),)
[5] ((dim=2|id=65|"S=1/2,Site,n=5"),)


Now the resulting 'MPS' only has the physical indices.

# Ground Truth Overlap

Now let's get the ground truth overlap using ITensors `inner` function:

In [6]:
gt_overlap = inner(phi, psi)

0.001855525188603962

Let's also get the squared norm:

In [7]:
gt_sq_norm_overlap = (abs(inner(phi, psi)))^2

3.442973725543769e-6

# Scheme

## Site 1

We begin with site 1 and do the following operation. We compute the state $\ket{\phi_1} = \braket{\Phi_1 | \psi}$ where $\bra{\Phi_1}$ is the data ket for site 1. Then we calculate $\braket{\phi_1 | \phi_1}$ to mimic the process of the MPO without explicitly constructing the outer product:

In [8]:
orthogonalize!(psi, 1); 

In [9]:
phi_1_psi_1 = phi[1] * psi[1] 
phi_1_psi_1_dag = prime(dag(phi_1_psi_1))
site_1_combined = phi_1_psi_1 * phi_1_psi_1_dag

ITensor ord=2 (dim=4|id=168|"Link,l=1") (dim=4|id=168|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

The result is the same as what we would have obtained if we used an MPO form constructed from $\ket{\psi}\bra{\psi}$ and contracted with the first data ket/bra.

### Trace

We compute the trace by contracting the ket version of a site with the bra version of itself. Because the physical indices are primed, we need to construct a delta tensor with the physical site indices:

In [10]:
delta(s[2], prime(s[2]))

ITensor ord=2 (dim=2|id=581|"S=1/2,Site,n=2") (dim=2|id=581|"S=1/2,Site,n=2")'
NDTensors.Diag{Float64, Float64}

In [11]:
s2_tr = psi[2] * prime(dag(psi[2])) * delta(s[2], prime(s[2]))

ITensor ord=4 (dim=4|id=168|"Link,l=1") (dim=4|id=12|"Link,l=2") (dim=4|id=168|"Link,l=1")' (dim=4|id=12|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

In [12]:
s3_tr = psi[3] * prime(dag(psi[3])) * delta(s[3], prime(s[3]))
s4_tr = psi[4] * prime(dag(psi[4])) * delta(s[4], prime(s[4]))
s5_tr = psi[5] * prime(dag(psi[5])) * delta(s[5], prime(s[5]));

Contract the traces together

In [13]:
R = s2_tr * s3_tr * s4_tr * s5_tr

ITensor ord=2 (dim=4|id=168|"Link,l=1") (dim=4|id=168|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [14]:
@show matrix(R)

matrix(R) = [1.0000000000000002 9.71445146547012e-17 3.3306690738754696e-16 6.938893903907228e-17; 9.71445146547012e-17 0.9999999999999999 -1.942890293094024e-16 -1.6653345369377348e-16; 3.3306690738754696e-16 -1.9949319973733282e-16 1.0000000000000007 -1.8735013540549517e-16; 6.938893903907228e-17 -1.942890293094024e-16 -1.8041124150158794e-16 1.0]


4×4 Matrix{Float64}:
 1.0           9.71445e-17   3.33067e-16   6.93889e-17
 9.71445e-17   1.0          -1.94289e-16  -1.66533e-16
 3.33067e-16  -1.99493e-16   1.0          -1.8735e-16
 6.93889e-17  -1.94289e-16  -1.80411e-16   1.0

Result is essentially a diagonal matrix of ones. We then contract this with the operation on the first site:

In [15]:
effective_overlap = site_1_combined * R

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [16]:
@show effective_overlap

effective_overlap = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.849436192448368


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

# Second Site

Repeating the same process as above but for the second site...

In [17]:
orthogonalize!(psi, 2)

MPS
[1] ((dim=2|id=567|"S=1/2,Site,n=1"), (dim=2|id=854|"Link,l=1"))
[2] ((dim=2|id=581|"S=1/2,Site,n=2"), (dim=4|id=12|"Link,l=2"), (dim=2|id=854|"Link,l=1"))
[3] ((dim=4|id=12|"Link,l=2"), (dim=2|id=944|"S=1/2,Site,n=3"), (dim=4|id=235|"Link,l=3"))
[4] ((dim=4|id=235|"Link,l=3"), (dim=2|id=489|"S=1/2,Site,n=4"), (dim=2|id=56|"Link,l=4"))
[5] ((dim=2|id=56|"Link,l=4"), (dim=2|id=65|"S=1/2,Site,n=5"))


In [18]:
phi_2_psi_2 = phi[2] * psi[2] 
phi_2_psi_2_dag = prime(dag(phi_2_psi_2))
site_2_combined = phi_2_psi_2 * phi_2_psi_2_dag

ITensor ord=4 (dim=4|id=12|"Link,l=2") (dim=2|id=854|"Link,l=1") (dim=4|id=12|"Link,l=2")' (dim=2|id=854|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [19]:
s1_tr = psi[1] * prime(dag(psi[1])) * delta(s[1], prime(s[1]))

ITensor ord=2 (dim=2|id=854|"Link,l=1") (dim=2|id=854|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [20]:
@show s1_tr

s1_tr = ITensor ord=2
Dim 1: (dim=2|id=854|"Link,l=1")
Dim 2: (dim=2|id=854|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 1.0  0.0
 0.0  1.0


ITensor ord=2 (dim=2|id=854|"Link,l=1") (dim=2|id=854|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [21]:
s3_tr = psi[3] * prime(dag(psi[3])) * delta(s[3], prime(s[3]))
s4_tr = psi[4] * prime(dag(psi[4])) * delta(s[4], prime(s[4]))
s5_tr = psi[5] * prime(dag(psi[5])) * delta(s[5], prime(s[5]))
L = s1_tr
R = s3_tr * s4_tr * s5_tr

ITensor ord=2 (dim=4|id=12|"Link,l=2") (dim=4|id=12|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

In [22]:
@show R

R = ITensor ord=2
Dim 1: (dim=4|id=12|"Link,l=2")
Dim 2: (dim=4|id=12|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}
 4×4
  0.9999999999999999      -1.6653345369377348e-16   5.551115123125783e-17   -5.551115123125783e-17
 -1.1102230246251565e-16   1.0000000000000002       1.6653345369377348e-16  -4.440892098500626e-16
  1.1102230246251565e-16   9.71445146547012e-17     1.0000000000000002       0.0
 -5.551115123125783e-17   -4.163336342344337e-16   -5.551115123125783e-17    1.0000000000000002


ITensor ord=2 (dim=4|id=12|"Link,l=2") (dim=4|id=12|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

In [23]:
site_2_combined

ITensor ord=4 (dim=4|id=12|"Link,l=2") (dim=2|id=854|"Link,l=1") (dim=4|id=12|"Link,l=2")' (dim=2|id=854|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}

In [24]:
effective_overlap2 = site_2_combined * R * L

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [25]:
@show effective_overlap2

effective_overlap2 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.3100227744825973


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

# Third Site

In [26]:
orthogonalize!(psi, 3)

MPS
[1] ((dim=2|id=567|"S=1/2,Site,n=1"), (dim=2|id=854|"Link,l=1"))
[2] ((dim=2|id=581|"S=1/2,Site,n=2"), (dim=2|id=854|"Link,l=1"), (dim=4|id=933|"Link,l=2"))
[3] ((dim=2|id=944|"S=1/2,Site,n=3"), (dim=4|id=235|"Link,l=3"), (dim=4|id=933|"Link,l=2"))
[4] ((dim=4|id=235|"Link,l=3"), (dim=2|id=489|"S=1/2,Site,n=4"), (dim=2|id=56|"Link,l=4"))
[5] ((dim=2|id=56|"Link,l=4"), (dim=2|id=65|"S=1/2,Site,n=5"))


In [27]:
phi_3_psi_3 = phi[3] * psi[3] 
phi_3_psi_3_dag = prime(dag(phi_3_psi_3))
site_3_combined = phi_3_psi_3 * phi_3_psi_3_dag

ITensor ord=4 (dim=4|id=235|"Link,l=3") (dim=4|id=933|"Link,l=2") (dim=4|id=235|"Link,l=3")' (dim=4|id=933|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

In [28]:
s1_tr = psi[1] * prime(dag(psi[1])) * delta(s[1], prime(s[1]))
s2_tr = psi[2] * prime(dag(psi[2])) * delta(s[2], prime(s[2]))
L = s1_tr * s2_tr

ITensor ord=2 (dim=4|id=933|"Link,l=2") (dim=4|id=933|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}

In [29]:
s4_tr = psi[4] * prime(dag(psi[4])) * delta(s[4], prime(s[4]))
s5_tr = psi[5] * prime(dag(psi[5])) * delta(s[5], prime(s[5]))
R = s4_tr * s5_tr

ITensor ord=2 (dim=4|id=235|"Link,l=3") (dim=4|id=235|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

In [30]:
effective_overlap3 = site_3_combined * R * L

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [31]:
@show effective_overlap3

effective_overlap3 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.0875063691710853


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

# Site 4

In [32]:
orthogonalize!(psi, 4)

MPS
[1] ((dim=2|id=567|"S=1/2,Site,n=1"), (dim=2|id=854|"Link,l=1"))
[2] ((dim=2|id=581|"S=1/2,Site,n=2"), (dim=2|id=854|"Link,l=1"), (dim=4|id=933|"Link,l=2"))
[3] ((dim=2|id=944|"S=1/2,Site,n=3"), (dim=4|id=933|"Link,l=2"), (dim=4|id=607|"Link,l=3"))
[4] ((dim=2|id=489|"S=1/2,Site,n=4"), (dim=2|id=56|"Link,l=4"), (dim=4|id=607|"Link,l=3"))
[5] ((dim=2|id=56|"Link,l=4"), (dim=2|id=65|"S=1/2,Site,n=5"))


In [33]:
phi_4_psi_4 = phi[4] * psi[4] 
phi_4_psi_4_dag = prime(dag(phi_4_psi_4))
site_4_combined = phi_4_psi_4 * phi_4_psi_4_dag

ITensor ord=4 (dim=2|id=56|"Link,l=4") (dim=4|id=607|"Link,l=3") (dim=2|id=56|"Link,l=4")' (dim=4|id=607|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

In [34]:
s1_tr = psi[1] * prime(dag(psi[1])) * delta(s[1], prime(s[1]))
s2_tr = psi[2] * prime(dag(psi[2])) * delta(s[2], prime(s[2]))
s3_tr = psi[3] * prime(dag(psi[3])) * delta(s[3], prime(s[3]))
L = s1_tr * s2_tr * s3_tr

ITensor ord=2 (dim=4|id=607|"Link,l=3") (dim=4|id=607|"Link,l=3")'
NDTensors.Dense{Float64, Vector{Float64}}

In [35]:
s5_tr = psi[5] * prime(dag(psi[5])) * delta(s[5], prime(s[5]))
R = s5_tr

ITensor ord=2 (dim=2|id=56|"Link,l=4") (dim=2|id=56|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [36]:
effective_overlap4 = site_4_combined * R * L

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [37]:
@show effective_overlap4

effective_overlap4 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.5721364438292501


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

# Site 5

In [38]:
orthogonalize!(psi, 5)

MPS
[1] ((dim=2|id=567|"S=1/2,Site,n=1"), (dim=2|id=854|"Link,l=1"))
[2] ((dim=2|id=581|"S=1/2,Site,n=2"), (dim=2|id=854|"Link,l=1"), (dim=4|id=933|"Link,l=2"))
[3] ((dim=2|id=944|"S=1/2,Site,n=3"), (dim=4|id=933|"Link,l=2"), (dim=4|id=607|"Link,l=3"))
[4] ((dim=2|id=489|"S=1/2,Site,n=4"), (dim=4|id=607|"Link,l=3"), (dim=2|id=74|"Link,l=4"))
[5] ((dim=2|id=65|"S=1/2,Site,n=5"), (dim=2|id=74|"Link,l=4"))


In [39]:
phi_5_psi_5 = phi[5] * psi[5] 
phi_5_psi_5_dag = prime(dag(phi_5_psi_5))
site_5_combined = phi_5_psi_5 * phi_5_psi_5_dag

ITensor ord=2 (dim=2|id=74|"Link,l=4") (dim=2|id=74|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [40]:
s1_tr = psi[1] * prime(dag(psi[1])) * delta(s[1], prime(s[1]))
s2_tr = psi[2] * prime(dag(psi[2])) * delta(s[2], prime(s[2]))
s3_tr = psi[3] * prime(dag(psi[3])) * delta(s[3], prime(s[3]))
s4_tr = psi[4] * prime(dag(psi[4])) * delta(s[4], prime(s[4]))
L = s1_tr * s2_tr * s3_tr * s4_tr

ITensor ord=2 (dim=2|id=74|"Link,l=4") (dim=2|id=74|"Link,l=4")'
NDTensors.Dense{Float64, Vector{Float64}}

In [41]:
effective_overlap5 = site_5_combined * L

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [42]:
@show effective_overlap5

effective_overlap5 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
1.0015797382041307


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [43]:
sum = effective_overlap + effective_overlap2 + effective_overlap3 + effective_overlap4 + effective_overlap5
@show sum/5

sum / 5 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.5641363036270863


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [44]:
effective_overlap_final = scalar(sum/5)

0.5641363036270863

# MPO Version

In [45]:
psi_mpo = outer(psi, psi);

┌ Warning: Calling `outer(ψ::MPS, ϕ::MPS)` for MPS `ψ` and `ϕ` with shared indices is deprecated. Currently, we automatically prime `ψ` to make sure the site indices don't clash, but that will no longer be the case in ITensors v0.4. To upgrade your code, call `outer(ψ', ϕ)`. Although the new interface seems less convenient, it will allow `outer` to accept more general outer products going forward, such as outer products where some indices are shared (a batched outer product) or outer products of MPS between site indices that aren't just related by a single prime level.
│ Stacktrace:
│   [1] backtrace()
│     @ Base ./error.jl:114
│   [2] warn_once(msg::String, funcsym::Symbol; force::Bool, stacktrace::Bool)
│     @ ITensors ~/.julia/packages/ITensors/WMeVS/src/utils.jl:8
│   [3] warn_once
│     @ ~/.julia/packages/ITensors/WMeVS/src/utils.jl:5 [inlined]
│   [4] #deprecate_make_inds_unmatch#977
│     @ ~/.julia/packages/ITensors/WMeVS/src/mps/mpo.jl:146 [inlined]
│   [5] deprecate_make_

In [46]:
psi_mpo

MPO
[1] ((dim=2|id=567|"S=1/2,Site,n=1")', (dim=2|id=567|"S=1/2,Site,n=1"), (dim=4|id=549|"Link,l=1"))
[2] ((dim=2|id=581|"S=1/2,Site,n=2")', (dim=2|id=581|"S=1/2,Site,n=2"), (dim=15|id=505|"Link,l=2"), (dim=4|id=549|"Link,l=1"))
[3] ((dim=2|id=944|"S=1/2,Site,n=3")', (dim=2|id=944|"S=1/2,Site,n=3"), (dim=16|id=905|"Link,l=3"), (dim=15|id=505|"Link,l=2"))
[4] ((dim=2|id=489|"S=1/2,Site,n=4")', (dim=2|id=489|"S=1/2,Site,n=4"), (dim=4|id=481|"Link,l=4"), (dim=16|id=905|"Link,l=3"))
[5] ((dim=2|id=65|"S=1/2,Site,n=5")', (dim=2|id=65|"S=1/2,Site,n=5"), (dim=4|id=481|"Link,l=4"))


Site 1

In [47]:
s1 = psi_mpo[1] * phi[1] * prime(dag(phi[1]))

ITensor ord=1 (dim=4|id=549|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

Trace over remaining sites

In [48]:
s2_tr = psi_mpo[2] * delta(s[2], prime(s[2]))
s3_tr = psi_mpo[3] * delta(s[3], prime(s[3]))
s4_tr = psi_mpo[4] * delta(s[4], prime(s[4]))
s5_tr = psi_mpo[5] * delta(s[5], prime(s[5]))
contracted_traces_s1 = s2_tr * s3_tr * s4_tr * s5_tr

ITensor ord=1 (dim=4|id=549|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [49]:
@show s1 * contracted_traces_s1

s1 * contracted_traces_s1 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.8494361619502766


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

site 2

In [50]:
s2 = psi_mpo[2] * phi[2] * prime(dag(phi[2]))

ITensor ord=2 (dim=15|id=505|"Link,l=2") (dim=4|id=549|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [51]:
s1_tr = psi_mpo[1] * delta(s[1], prime(s[1]))

ITensor ord=1 (dim=4|id=549|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [52]:
contracted_traces_s2 = s1_tr * s3_tr * s4_tr * s5_tr

ITensor ord=2 (dim=4|id=549|"Link,l=1") (dim=15|id=505|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [53]:
@show s2 * contracted_traces_s2

s2 * contracted_traces_s2 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.3100227705681342


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

site 3

In [54]:
s3 = psi_mpo[3] * phi[3] * prime(dag(phi[3]))

ITensor ord=2 (dim=16|id=905|"Link,l=3") (dim=15|id=505|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [55]:
contracted_traces_s3 = s1_tr * s2_tr * s4_tr * s5_tr

ITensor ord=2 (dim=15|id=505|"Link,l=2") (dim=16|id=905|"Link,l=3")
NDTensors.Dense{Float64, Vector{Float64}}

In [56]:
@show s3 * contracted_traces_s3

s3 * contracted_traces_s3 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.08750636830667217


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

site 4

In [57]:
s4 = psi_mpo[4] * phi[4] * prime(dag(phi[4]))

ITensor ord=2 (dim=4|id=481|"Link,l=4") (dim=16|id=905|"Link,l=3")
NDTensors.Dense{Float64, Vector{Float64}}

In [58]:
contracted_traces_s4 = s1_tr * s2_tr * s3_tr * s5_tr

ITensor ord=2 (dim=16|id=905|"Link,l=3") (dim=4|id=481|"Link,l=4")
NDTensors.Dense{Float64, Vector{Float64}}

In [59]:
@show s4 * contracted_traces_s4

s4 * contracted_traces_s4 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.5721364351860072


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

site 5

In [60]:
s5 = psi_mpo[5] * phi[5] * prime(dag(phi[5]))

ITensor ord=1 (dim=4|id=481|"Link,l=4")
NDTensors.Dense{Float64, Vector{Float64}}

In [61]:
contracted_traces_s5 = s1_tr * s2_tr * s3_tr * s4_tr

ITensor ord=1 (dim=4|id=481|"Link,l=4")
NDTensors.Dense{Float64, Vector{Float64}}

In [62]:
@show s5 * contracted_traces_s5

s5 * contracted_traces_s5 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
1.001579706661465


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [63]:
sum = s5 * contracted_traces_s5 + s4 * contracted_traces_s4 + s3 * contracted_traces_s3 + s2 * contracted_traces_s2 + s1 * contracted_traces_s1

ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [64]:
@show sum/5

sum / 5 = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
0.564136288534511


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

# Validation of "Effective Overlap"

Approach:
- Take in a trained MPS where we know the value of the overlaps with the trained set are large.
- Using the same training samples, compute the effective overlap.
- Compare the overlap with the effective overlap. Do they represent the same thing, that is, the similarity between the state represented by the MPS and the product state representation
of the training data.

Part 2 (benchmarking):
- Quantify the performance of the effective overlap i.e. how long does it take to compute verus ITensors built-in overlap function.
- Scaling -> how does the time taken to compute scale with the number of sites, $N$ and the local dimension $d$?